# Prompt

千帆提供了 Prompt 管理功能，可以快速地使用平台预置的优质 Prompt，或者保存用户自定义的 Prompt。SDK 也为用户快速使用 Prompt 提供了辅助。

Prompt 相关功能需要使用 Access Key 和 Secret Key 进行鉴权，获取方式参见 [文档](https://cloud.baidu.com/doc/Reference/s/9jwvz2egb)。

In [1]:
import os
import qianfan

os.environ["QIANFAN_ACCESS_KEY"] = "your access key"
os.environ["QIANFAN_SECRET_KEY"] = "your secret key"

可以通过这一方式导入 Prompt

In [1]:
from qianfan.common import Prompt

## 快速使用

平台上预置的 Prompt 以及用户自定义的模型都可以在 [千帆控制台](https://console.bce.baidu.com/qianfan/prompt/template) 获得，之后可以在 SDK 中用 Prompt 的名称快速获取 Prompt 对象。

加载平台上的相关资源需要使用 hub 功能。

In [1]:
from qianfan.common.hub import hub

p = hub.load("prompt/区域美食推荐")

之后就可以调用 `render` 方法对 Prompt 模版进行填充，得到最终的文本。

In [2]:
# 第二个参数是 negative prompt，文生文场景下为空，所以用 _ 省略
prompt, _ = p.render(region="上海")
print(prompt)

作为游客，告诉我上海必吃的10大美食


也可以通过如下方式获得 Prompt 的一些元信息

In [3]:
print(p.template)
print(p.variables)

作为游客，告诉我{region}必吃的10大美食
['region']


SDK 也支持文生图类型的 Prompt，可以通过文生图类的 Prompt 名称进行初始化

In [4]:
txt2img_prompt = hub.load("prompt/角色设计")

prompt, negative_prompt = txt2img_prompt.render()
print(prompt)
print(negative_prompt)

white hair female, close up character design, multiple concept designs, concept design sheet, white background, style of Yoshitaka Amano
(worst quality, low quality:1.4), deformed iris, deformed pupils, (deformeddistorted, disfigured:1.3), cropped, out of frame.poorly drawn, bad anatomy, wrong anatomy, extralimb, missing limb, floating limbs


Prompt 的场景可以通过 `scene_type` 属性获得

In [5]:
from qianfan.consts import PromptSceneType

txt2img_prompt.scene_type == PromptSceneType.Text2Image

True

借助 Prompt，我们就可以快速地完成一些任务，比如：

In [7]:
import qianfan

p = hub.load("prompt/区域美食推荐")

def recommend_food(region):
    prompt, _ = p.render(region=region)
    r = qianfan.Completion().do(prompt)
    return r['result']

print(recommend_food("上海"))

作为游客，上海必吃的十大美食包括：

1. **生煎馒头**：上海特色早餐，鲜肉馅，底部脆硬，整体味道鲜美。
2. **小笼包**：上海的小笼包以汤汁丰足、味道醇正著称。
3. **蟹壳黄**：上海本地特色小吃，以烤至酥脆的饼皮和咸蛋黄肉馅为主。
4. **白斩鸡**：最原汁原味的上海白斩鸡搭配甜酱是最佳的选择。
5. **糖醋排骨**：使用传统的糖醋工艺制作的上海特色菜肴，酸甜可口。
6. **炸猪排**：与排骨相似的上海特色小吃，通常与番茄酱一起食用。
7. **清炒虾仁**：上海的传统名菜，口感鲜美，营养丰富。
8. **响油鳝糊**：上海本地的特色鳝鱼美食，味道鲜美，做法相对复杂。
9. **三鲜馄饨**：上海的传统小吃，以鲜肉、虾仁和海鲜为主要馅料，味道鲜美。
10. **糟卤**：一种特殊的调味料，可以增加食物的鲜美程度，尤其适合海鲜类食品。

以上十种美食涵盖了甜、咸、酥、脆、鲜等多种口感，可以满足不同游客的口味。在品尝美食的同时，也可以搭配上海本地的茶点，体验上海的饮食文化。


## 本地 Prompt

如果不希望使用平台上的模型，只希望本地尝试，也可以通过 Prompt 对象实现，初始化时仅传入 `template` 字段即可。

In [10]:
p = Prompt("本地 prompt {var1}")

prompt, _ = p.render(var1="hello")
print(prompt)

本地 prompt hello


还可以传入更多参数，创建一个更为复杂的本地 Prompt，比如下面创造了一个场景为文生图的 Prompt，并且给定了分别设置了正向和负向 Prompt，并且采用 `(())` 作为标识符。

In [11]:
prompt = Prompt(
    name="txt2img",
    template="txt2img template ((v1))",
    scene_type=PromptSceneType.Text2Image,
    negative_template="negative template ((v3))",
    identifier="(())",
)

## 上传/更新 Prompt

对于本地 Prompt，还可以使用 hub 将其上传保存至平台，方便后续快速使用，上传仅需要利用 `hub.push` 即可。关于 hub 的一些其他操作，可以参考 hub 的相关文档。

In [12]:
p = Prompt(
    template="本地 prompt {var1}",
    # 对于平台上的 prompt 来说，name 是必须的，因此上传前必须先设置
    name="cookbook_prompt"
)

# 也可以通过这种方式设置 name
p.name = "cookbook_prompt"
hub.push(p)

后续还可以本地对 Prompt 进行更新，并继续用 `hub.push` 方法更新至平台上。

In [13]:
p.set_template("新的 Prompt {new_var}")
hub.push(p)

print(p.variables)
print(p.render(new_var="hello"))

['new_var']
('新的 Prompt hello', None)


## 删除 Prompt

SDK 提供了 `delete` 方法，可以快速删除平台上的 Prompt。

In [14]:
p = hub.load("prompt/cookbook_prompt")
p.delete()

如果没有抛出异常，那么这条 Prompt 已经从平台上移除。

## 保存/读取 Prompt

通常在进行 Prompt 调优后，需要保存 Prompt 以便后续使用，SDK 也提供了 `save_to_file` 方法，可以将 Prompt 保存保存至本地。 

In [12]:
p = Prompt(template="这是一个用于{usage}的 Prompt")
p.save_to_file("test_prompt.tpl")

而在再次使用时，只需要通过 `from_file` 方法即可读取 Prompt。

如果希望调整 Prompt，可以直接对模版文件修改，从而既可以避免在代码中出现冗长的模版字符串，也可以因调整模版而反复修改代码。

In [13]:
p = Prompt.from_file("test_prompt.tpl")
prompt, _ = p.render(usage="测试")
print(prompt)

这是一个用于测试的 Prompt


## 优化 Prompt

为了能够提升模型的生成效果，SDK 提供了优化 prompt 的功能，能够快速生成高质量的 prompt。

In [3]:
old_prompt = Prompt("帮我写一份{job}的年终报告")
new_prompt = old_prompt.optimize()
print(new_prompt.template)

请撰写一份{job}的年终报告，包括该{job}的概要、成绩和挑衅。请在报告中提供{job}的详细说明和目标，并阐明{job}的成果和挑战。同时，请提供相关数据和指标，以证明{job}的有效性和可操作性。请注意，您的报告应该简洁明了，并包含所有必要的细节和信息，以帮助读者更好地理解{job}的表现和结果。


优化可以开启不同选项：

- `optimize_quality`：优化 prompt 质量
- `simplify_prompt`：简化 prompt，可以省去“的”、“吧”等含义不强的文本实体，精炼语料内容并降低推理成本
- `iteration_round`：优化迭代轮数，默认为 1
- `enable_cot`：开启思维链，将指引模型拆解Prompt内容，逐步进行推理。建议仅在数学计算、逻辑推理等场景下开启使用。

或者也可以给定一个样例数据集，调用模型对 prompt 进行优化。

In [2]:
from qianfan.dataset import Dataset

dataset = Dataset.load(
    data_file="./essay.jsonl",
    organize_data_as_group=False, 
    input_columns=["question", "article"], 
    reference_column="expect"
)

old_prompt = p = Prompt(
    """请帮我批改如下高考作文，从内容、标答、发展等级、字数、缺点和改进意见几个方面给出评价和评分（0-70分）：
题目：{question}
作文内容：{article}"""
)
new_prompt = old_prompt.optimize_by_example(example=dataset)

[INFO] [04-10 17:34:43] dataset.py:389 [t:139889838559488]: no data source was provided, construct
[INFO] [04-10 17:34:43] dataset.py:257 [t:139889838559488]: construct a file data source from path: ./essay.jsonl, with args: {'input_columns': ['question', 'article'], 'reference_column': 'expect'}
[INFO] [04-10 17:34:43] file.py:280 [t:139889838559488]: use format type FormatType.Jsonl
[INFO] [04-10 17:34:43] utils.py:331 [t:139889838559488]: need create cached arrow file for /root/work/bce-qianfan-sdk/cookbook/essay.jsonl
[INFO] [04-10 17:34:43] utils.py:376 [t:139889838559488]: start to write arrow table to /root/.qianfan_cache/dataset/root/work/bce-qianfan-sdk/cookbook/essay.arrow
[INFO] [04-10 17:34:43] utils.py:388 [t:139889838559488]: writing succeeded
[INFO] [04-10 17:34:43] utils.py:262 [t:139889838559488]: start to get memory_map from /root/.qianfan_cache/dataset/root/work/bce-qianfan-sdk/cookbook/essay.arrow
[INFO] [04-10 17:34:43] utils.py:237 [t:139889838559488]: has got a m

In [3]:
print(new_prompt.template)


我现在需要你按照以下要求对一篇高考作文进行详细的批改和评价：

作文题目：{question}
作文内容：{article}

批改和评价要求：

1. **内容层面**：
   - 评估作文是否紧密围绕题目中心进行撰写。
   - 判断作文是否明确表达了中心思想，并且内容充实、有深度。

2. **表达层面**：
   - 分析作文的结构是否清晰，逻辑是否连贯。
   - 检查作文的语言表达是否流畅，有无明显的错别字、语法错误或表达不清的地方。

3. **发展等级与创新性**：
   - 判断作文是否有独特的观点或见解，是否能够引发读者的思考。
   - 评估作文在内容或表达上是否有创新之处，是否给读者留下深刻印象。

4. **具体建议与改进方向**：
   - 指出作文中需要改进的具体方面，如论证不够充分、例子不够贴切等。
   - 提供具体的改进建议，帮助作者提升作文的整体质量。

最后，请给出作文的总体评价，包括优点、不足以及你的总体印象。



## 评估 Prompt

在优化完 Prompt 后，SDK 提供了 `evaluate` 方法，可以快速评估不同 Prompt 的质量。

In [4]:
from qianfan import Completion

prompts = [old_prompt, new_prompt]
scenes = [
    {
        "args": {"job": "程序员"},
        "expected": "代码"
    },
    {
        "args": {"job": "产品经理"},
        "expected": "用户量"
    }
]
model = Completion()

results = Prompt.evaluate(prompts, scenes, model)

results 是一个 list，每个元素是一个 PromptEvaluateResult 对象，代表一个 Prompt 的评估结果。可以用 summary 字段获取评估总结。

In [6]:
for scene, result in zip(scenes, results):
    print(scene["args"]["job"])
    print(result.summary)
    print("======")

程序员
【用户期望结果】：

1. 清晰明确的提示词：用户需要一个简洁、明确的提示词，以便快速理解需要完成的任务。
2. 具体性：用户期望提示词能够提供足够的具体信息，以便生成符合要求的年终报告。
3. 通用性：用户希望提示词具有一定的通用性，以便能够适应不同的工作场景和需求。

【模型生成结果】：

1. 清晰明确：提示词“帮我写一份{job}的年终报告”简洁明了，用户可以快速理解需要完成的任务。
2. 具体性：该提示词提供了“job”这一关键词，用户可以根据自己的实际工作情况填写具体的职业或岗位名称，使得生成的年终报告更具针对性。
3. 通用性：该提示词中的“{job}”是一个占位符，可以根据不同的工作场景和需求进行替换，具有一定的通用性。

【评价】：

该提示词模版撰写得相对较好，既简洁明了，又具有一定的具体性和通用性，能够满足用户的需求。如果一定要对模版进行改进，可以考虑增加一些细节方面的提示，例如在“年终报告”前加上“工作总结”等关键词，以更好地引导用户生成符合要求的年终报告。
产品经理
根据您提供的模板，以下是对提示词模版撰写质量的评价：

1. 清晰度：模版使用了简洁明了的语言，使得用户可以快速理解要求的内容和格式。每个部分都有明确的标题，如“概要”、“成绩”和“挑衅”，这有助于用户组织思路和信息。
2. 适应性：模版使用了灵活的占位符，如“{job}”，使得该模版可以适用于多种不同的职位和场景。通过替换占位符，用户可以轻松地生成特定于其职位的年终报告。
3. 完整性：模版要求包含所有必要的部分，如“概要”、“成绩”和“挑衅”，以确保报告的全面性。这有助于用户在撰写报告时不会遗漏任何重要信息。
4. 数据和指标要求：模版强调了提供相关数据和指标的重要性，这有助于证明该职位的有效性和可操作性。这是年终报告中一个非常关键的部分，因为它为读者提供了具体的信息，以评估该职位的成果和挑战。
5. 简洁性：模版提醒用户报告应该简洁明了，这有助于确保报告易于阅读和理解。在撰写报告时，避免冗余和不必要的细节是非常重要的，模版的这一要求有助于实现这一目标。

综上所述，该提示词模版撰写质量较高，能够为用户提供清晰、完整和实用的指导，以确保他们能够生成一份出色的年终报告。


同时也可以通过 scene 字段，获取不同场景下每个 prompt 评估分数与模型输出。

In [14]:
scene_list = results[1].scene
for scene in scene_list:
    print("预期输出：",scene['expected_target'])
    print("评价得分：", scene['score'])
    print("模型输出：", scene['response'])
    print("======")
    

预期输出： 代码
评价得分： 0.6875922409817495
模型输出： 年度工作总结与展望

尊敬的领导和各位同事：

我是XXX，一名普通的程序员，很荣幸在这里向大家汇报我今年的工作情况。

一、个人概要

我从事软件开发工作已有X年，对各种编程语言如Python、Java等有着深厚的功底，同时对数据结构和算法有深入的理解。我的目标是不断提升自己的技术水平，为公司的业务发展提供强有力的支持。

二、成绩和挑衅

今年，我参与了多个项目的开发，其中最值得一提的是我们团队成功完成了“XX系统”的升级工作。在这个项目中，我负责了核心部分的编程和优化，通过改进算法和提升效率，我们成功地为客户系统带来了20%的性能提升，得到了客户和领导的高度赞扬。

然而，我也面临着一些挑战。一方面，随着技术的快速发展，我需要不断学习新的知识和技能，以应对不断变化的开发环境；另一方面，我注意到团队中一些成员对复杂技术的接受程度有限，这在一定程度上制约了我们项目的进展。为此，我感到有必要提升自己的教学和传播能力，以便更好地帮助团队成员提升技术水平。

三、程序员的详细说明和目标

1. 编程语言和工具：我主要使用Python和Java进行开发，工具包括Git、Jira等项目管理工具，以及各种数据库和云服务。
2. 技能提升：今年我特别关注了人工智能和机器学习的相关知识，希望能将这些技术应用到我们的项目中。同时，我也提高了我的英语阅读能力，以便更好地获取国际前沿的技术信息。
3. 目标：我希望在明年能够带领我们的团队完成更多的高质量项目，同时提升我们的技术教学能力，帮助更多的团队成员提升技术水平。

四、成果和挑战

今年，我们团队取得了丰硕的成果。我们参与的多个项目都得到了客户的满意反馈，其中“XX系统”的升级项目更是我们的亮点。我们的代码质量得到了显著提升，错误率下降了30%，这大大提高了我们项目的成功率。

然而，我们也面临一些挑战。首先，随着业务的发展，我们的项目数量和复杂度都在增加，这给我们团队带来了很大的工作压力。其次，尽管我们在代码质量和效率上有了一定的提升，但在某些特殊场景下的性能优化仍然是我们需要解决的问题。最后，如何培养和提升团队成员的技术能力也是一个我们需要面对的挑战。

五、数据和指标

为了证明我们团队的有效性和可操作性，我附上了一些相关的数据和指标：

1.